In [4]:
import sys
import time
import duckdb as ddb
from colory.color import Color
from datetime import datetime

data = ddb.read_parquet("../week3/rplace.parquet")

most_placed = ddb.sql(f"""
    SELECT x, y, COUNT(*) as count
    FROM data
    GROUP BY x, y
    ORDER BY count DESC
    LIMIT 3
""").fetchall()

print(most_placed)



[(0, 0, 98807), (359, 564, 69198), (349, 564, 55230)]


As show by the above query on the r/place dataset, the top 3 most placed pixels and their counts are as follows:
1. (0, 0)     98,807 times
2. (359, 564) 69,198 times
3. (349, 564) 55,230 times

In [ ]:
most_placed = ddb.sql(f"""
    SELECT x, y, COUNT(*) as count
    FROM data
    GROUP BY x, y
    ORDER BY count DESC
    LIMIT 3
""").fetchall()

print(most_placed)